# Introdução
Este exemplo mostra como usar técnicas de NLP (Natural Language Processing) para analisar textos, extrair insights, classificar sentimentos e transformar textos não estruturados em dados estruturados úteis. Essas técnicas podem ser aplicadas a diversos casos de uso, como análise de sentimentos de produtos, feedback de clientes, e muito mais

# Passo 1: Carregar e preparar os dados
Para este exemplo, será utilizado um conjunto de dados fictício de análises de produtos.

In [16]:
import pandas as pd
# Criação de um DataFrame fictício com análises de produtos
data = {
    'review': [
        'This product is great! I loved it.',
        'Terrible customer service. I am very disappointed.',
        'The quality is fantastic. Highly recommended!',
        'Not worth the money. Very poor quality.',
        'I am extremely satisfied with this purchase.'
    ]
}

df = pd.DataFrame(data)
print("Dados carregados:")
print(df)

Dados carregados:
                                              review
0                 This product is great! I loved it.
1  Terrible customer service. I am very disappoin...
2      The quality is fantastic. Highly recommended!
3            Not worth the money. Very poor quality.
4       I am extremely satisfied with this purchase.


# Passo 2: Pré-processar os Textos
Limpando os textos convertendo para minúsculas, removendo pontuações, tokenizando, removendo stopwords e lematizando.

In [17]:
import nltk # é uma das bibliotecas mais populares e poderosas para processamento de linguagem natural (NLP) em Python
from nltk.corpus import stopwords # módulo do nlkt que contém uma lista de palavras comuns que geralmente são removidas durante o pré-processamento de texto em tarefas de NLP
from nltk.tokenize import word_tokenize # módulo do nlkt que é usada para dividir um texto em palavras individuais (tokens)
from nltk.stem import WordNetLemmatizer # módulo do nlkt que é usada para reduzir palavras à sua forma base ou raiz (lemmas)
import string # contém diversas constantes e classes úteis para manipulação de strings

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Converter o texto para minúsculas
    text = text.lower()
    # Remover pontuações
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenizar o texto
    tokens = word_tokenize(text)
    # Remover stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lematizar as palavras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Juntar as palavras de volta em uma string
    text = ' '.join(tokens)
    return text

# Aplicar a função de pré-processamento
df['cleaned_review'] = df['review'].apply(preprocess_text)
print("\nTextos pré-processados:")
print(df['cleaned_review'])


Textos pré-processados:
0                       product great loved
1    terrible customer service disappointed
2      quality fantastic highly recommended
3                  worth money poor quality
4              extremely satisfied purchase
Name: cleaned_review, dtype: object


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Passo 3: Classificar sentimentos

In [18]:
from textblob import TextBlob # fornece uma API simples para tarefas comuns de NLP, como análise de sentimentos, tradução, correção gramatical, tokenização, e muito mais
def classify_sentiment(text):
    analysis = TextBlob(text)
    # Classificar como positivo, negativo ou neutro
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Aplicar a função de classificação de sentimentos
df['sentiment'] = df['cleaned_review'].apply(classify_sentiment)
print("\nClassificação de sentimentos:")
print(df[['review', 'sentiment']])


Classificação de sentimentos:
                                              review sentiment
0                 This product is great! I loved it.  Positive
1  Terrible customer service. I am very disappoin...  Negative
2      The quality is fantastic. Highly recommended!  Positive
3            Not worth the money. Very poor quality.  Negative
4       I am extremely satisfied with this purchase.  Positive


# Passo 4: Extrair Insights

In [19]:
from collections import Counter # módulo do Python que fornece ferramentas para trabalhar com coleções
# Contagem de palavras
all_words = ' '.join(df['cleaned_review']).split()
word_freq = Counter(all_words)
print("\nFrequência de palavras:")
print(word_freq.most_common(5))

# Contagem de sentimentos
sentiment_counts = df['sentiment'].value_counts()
print("\nContagem de sentimentos:")
print(sentiment_counts)


Frequência de palavras:
[('quality', 2), ('product', 1), ('great', 1), ('loved', 1), ('terrible', 1)]

Contagem de sentimentos:
sentiment
Positive    3
Negative    2
Name: count, dtype: int64


# Passo 5: Transformar textos em dados estruturados

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer # módulo do scikit-learn que é usado para extrair recursos de texto
# Transformar textos em uma representação vetorial TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_review'])

# Mostrar a matriz TF-IDF
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print("\nMatriz TF-IDF:")
print(tfidf_df)


Matriz TF-IDF:
   customer  disappointed  extremely  fantastic    great    highly    loved  \
0       0.0           0.0    0.00000   0.000000  0.57735  0.000000  0.57735   
1       0.5           0.5    0.00000   0.000000  0.00000  0.000000  0.00000   
2       0.0           0.0    0.00000   0.523358  0.00000  0.523358  0.00000   
3       0.0           0.0    0.00000   0.000000  0.00000  0.000000  0.00000   
4       0.0           0.0    0.57735   0.000000  0.00000  0.000000  0.00000   

      money      poor  product  purchase   quality  recommended  satisfied  \
0  0.000000  0.000000  0.57735   0.00000  0.000000     0.000000    0.00000   
1  0.000000  0.000000  0.00000   0.00000  0.000000     0.000000    0.00000   
2  0.000000  0.000000  0.00000   0.00000  0.422242     0.523358    0.00000   
3  0.523358  0.523358  0.00000   0.00000  0.422242     0.000000    0.00000   
4  0.000000  0.000000  0.00000   0.57735  0.000000     0.000000    0.57735   

   service  terrible     worth  
0      

In [23]:
from google.colab import files
# Salvar o DataFrame como um arquivo CSV
tfidf_df.to_csv('matriz-tf-idf.csv', index=True)

In [24]:
# Baixar o arquivo CSV para pasta local
files.download('matriz-tf-idf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>